# Structuration DataSet (A ne plus relancer)

In [ ]:
# Mount GDrive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# On concaténe les 6 fichiers
import pandas as pd
df_0 = pd.read_csv("/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/emaildata_100000_0.csv")
df_1 = pd.read_csv("/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/emaildata_100000_1.csv")
df_2 = pd.read_csv("/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/emaildata_100000_2.csv")
df_3 = pd.read_csv("/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/emaildata_100000_3.csv")
df_4 = pd.read_csv("/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/emaildata_100000_4.csv")
df_5 = pd.read_csv("/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/emaildata_100000_5.csv")
# df = pd.read_csv("/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/enron.csv", nrows=1500)
frames = [df_0,df_1,df_2,df_3,df_4,df_5]
df = pd.concat(frames)
# la selection ci-dessous a permis de détecter 3 doublons (toutes colonnes identiques)
# df = df.loc[(df['subject'] == "Sagewood Town Homes")]
print(df.shape)
# df = df_emails.loc[(df['subject'] == "Sagewood Town Homes")]
df.drop('Unnamed: 0', axis=1, inplace=True)
# suppression des doublons (sur toutes les colonnes sauf index)
df.drop_duplicates(subset=['date','sender', 'recipient1', 'subject','text'],keep = 'first', inplace=True)
# on renomme les colonnes
df = df.rename(columns={'sender': 'from', 'recipient1': 'to','subject': 'header', 'text': 'body'})
# et export en csv du résultat.
file_name = "/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/enron_unique.csv"
df.to_csv(file_name, encoding='utf-8', index=False)
print(df.shape)
# Taille du fichier divisée par 2 environ

Mounted at /content/drive
(405968, 6)
(227032, 5)


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 227032 entries, 0 to 11736
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   date    227032 non-null  object
 1   from    227032 non-null  object
 2   to      227032 non-null  object
 3   header  218946 non-null  object
 4   body    227032 non-null  object
dtypes: object(5)
memory usage: 10.4+ MB


# Import - Data Preprocessing structures

Mount Gdrive files

In [ ]:
# Mount GDrive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


Import Spacy

In [ ]:
#!pip install -U spacy
#!python -m spacy download en_core_web_sm
!pip install spacy-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 51 kB 244 kB/s 
     |████████████████████████████████| 4.2 MB 9.6 MB/s 
     |████████████████████████████████| 1.1 MB 45.1 MB/s 
     |████████████████████████████████| 6.6 MB 41.1 MB/s 
     |████████████████████████████████| 596 kB 71.3 MB/s 
     |████████████████████████████████| 101 kB 11.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
stopwords=list(STOP_WORDS)
from string import punctuation
punctuation=punctuation+ '\n'

In [ ]:
import pandas as pd
# Pour limiter les temps de calcul on travaille uniquement sur les 'nrows' premiers mails
df = pd.read_csv("/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/enron_unique.csv", nrows=200)
df.head()

,date,from,to,header,body
0,2001-05-14 16:39:00-07:00,phillip.allen@enron.com,tim.belden@enron.com,NaN,"['', 'Here is our forecast', '', ' ']"
1,2001-05-04 13:51:00-07:00,phillip.allen@enron.com,john.lavorato@enron.com,Re:,"['', 'Traveling to have a business meeting tak..."
2,2000-10-18 03:00:00-07:00,phillip.allen@enron.com,leah.arsdall@enron.com,Re: test,"['', 'test successful. way to go!!!']"
3,2000-10-23 06:13:00-07:00,phillip.allen@enron.com,randall.gay@enron.com,NaN,"['', 'Randy,', '', ' Can you send me a schedul..."
4,2000-08-31 05:07:00-07:00,phillip.allen@enron.com,greg.piper@enron.com,Re: Hello,"['', ""Let's shoot for Tuesday at 11:45. ""]"


Overview dataset

In [ ]:
# nbre d'expéditeurs distincts
len(df['from'].unique())

1

In [ ]:
# nbre de destinataires distincts
len(df['to'].unique())

88

Rajout colonnes pour traitements ultérieurs

In [ ]:
import numpy as np
df = df.replace('NaN', np.NaN)
# df_emails = df[df['header'].notna()]
df_emails = df
df_emails['cc'] = ''                    # pour identfication des personnes en copie
df_emails['NER_header'] = ''            # Spacy NER
df_emails['NER_body'] = ''              # Spacy NER
df_emails['body_clean'] = ''            # body sans caracteres parasites
df_emails['body_dict'] = ''             # body tokenisé
df_emails['summary_TFIDF'] = ''         # extractive summary by TF-IDF
df_emails['summary_spacy'] = ''         # extractive summary by Spacy
df_emails['summary_abstractive'] = ''   # abstractive summary by HugginFace/BERT
df_emails.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   date                 200 non-null    object
 1   from                 200 non-null    object
 2   to                   200 non-null    object
 3   header               139 non-null    object
 4   body                 200 non-null    object
 5   cc                   200 non-null    object
 6   NER_header           200 non-null    object
 7   NER_body             200 non-null    object
 8   body_clean           200 non-null    object
 9   body_dict            200 non-null    object
 10  summary_TFIDF        200 non-null    object
 11  summary_spacy        200 non-null    object
 12  summary_abstractive  200 non-null    object
dtypes: object(13)
memory usage: 20.4+ KB


In [ ]:
df_emails.columns

Index(['date', 'from', 'to', 'header', 'body', 'cc', 'NER_header', 'NER_body',
       'body_clean', 'body_dict', 'summary_TFIDF', 'summary_spacy',
       'summary_abstractive'],
      dtype='object')

In [ ]:
#
# A ce stade le fichier a la structure nécessaire à Melusine pour le nettoyage
#

# NER with Spacy

In [ ]:
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_sm')
spacy_stopwords = nlp.Defaults.stop_words
stopwords = spacy_stopwords
print(spacy_stopwords)


deb = 0
fin = df_emails.shape[0]
for x in range(deb,fin):
# subject
  subj = df_emails.iloc[x]['header']
#  print(x,' - ',subj)
  doc_subj = nlp(str(subj))
  df_emails['NER_header'][x] = doc_subj.ents

# body
  subj = df_emails.iloc[x]['body']
#  print(x,' - ',subj)
  doc_subj = nlp(str(subj))
  df_emails['NER_body'][x] = doc_subj.ents

{'full', 'to', 'hereafter', 'with', 'across', 'you', 'has', '’m', 'even', 'ever', 'mostly', 'could', 'almost', 'few', 'sometimes', 'just', 'whom', 'everyone', 'these', 'do', 'ours', 'very', 'seem', 'bottom', 'yourselves', 'otherwise', '’ve', 'whatever', 'he', 'of', 'moreover', 'ten', 'myself', 'whence', 'within', 'perhaps', 'therefore', 'than', 'forty', 'other', 'did', 'regarding', 'sixty', 'thus', 'call', 'no', "'d", 'which', 'none', 'seems', 'latterly', '‘s', "'re", 'onto', 'if', 'from', 'the', 'first', 'whole', 'whether', 'whereby', 'toward', 'off', '’s', 'indeed', 'whereafter', 'into', 'where', '‘ll', 'five', 'neither', 'afterwards', 'been', 'own', 'my', 'due', 'below', 'himself', 'whenever', 'along', 'on', 'nothing', 'twelve', 'can', 'besides', 'every', '‘d', 'third', '’d', 'however', 'becoming', 'thence', 'them', 'always', 'nevertheless', 'between', 'up', 'nine', 'mine', 'too', '’ll', 'per', 'various', 'made', 'would', 'beside', 'get', 'make', 'elsewhere', 'four', 'amongst', 'she

In [ ]:
df_emails[['header','NER_header','NER_body']][0:40]

,header,NER_header,NER_body
0,NaN,(),()
1,Re:,(),"((Austin),)"
2,Re: test,(),()
3,NaN,(),"((Randy), (Patti, S))"
4,Re: Hello,(),"((Tuesday), (11:45))"
5,Re: Hello,(),"((next, Tuesday), (Thursday))"
6,NaN,(),"((Phillip, Allen), (Mike, Grigsby), (Keith, Ho..."
7,Re: PRC review - phone calls,"((PRC),)","((10, and, 11:30, '),)"
8,Re: High Speed Internet Access,(),"((ISP), (2), (IP), (DNS), (3))"
9,FW: fixed forward or other Collar floor gas pr...,"((FW, :), (Collar))","((Phillip, K, Allen, /, HOU, /, ECT), (01:42, ..."


In [ ]:
id = 12
doc= nlp(df_emails['header'][id])
displacy.render(doc, style = "ent",jupyter = True)
print("----- eMail ",id," -----")
doc= nlp(df_emails['body'][id])
displacy.render(doc, style = "ent",jupyter = True)
print("\n")

----- eMail  12  -----


In [ ]:
# A ce stade le fichier a la structure attendue par melusine + l'identification des NER par Spacy
df_emails[['header','NER_header','body','NER_body']].iloc[id]

header              Consolidated positions: Issues & To Do list
NER_header                               ((Issues, &, To, Do),)
body          ['', '---------------------- Forwarded by Phil...
NER_body      ((Phillip, K, Allen, /, HOU, /, ECT), (02:16, ...
Name: 12, dtype: object

In [ ]:
df_emails[['header','body','body_dict','summary_TFIDF','NER_body']][0:30]

,header,body,body_dict,summary_TFIDF,NER_body
0,NaN,"['', 'Here is our forecast', '', ' ']",,,()
1,Re:,"['', 'Traveling to have a business meeting tak...",,,"((Austin),)"
2,Re: test,"['', 'test successful. way to go!!!']",,,()
3,NaN,"['', 'Randy,', '', ' Can you send me a schedul...",,,"((Randy), (Patti, S))"
4,Re: Hello,"['', ""Let's shoot for Tuesday at 11:45. ""]",,,"((Tuesday), (11:45))"
5,Re: Hello,"['', 'Greg,', '', ' How about either next Tues...",,,"((next, Tuesday), (Thursday))"
6,NaN,"['', 'Please cc the following distribution lis...",,,"((Phillip, Allen), (Mike, Grigsby), (Keith, Ho..."
7,Re: PRC review - phone calls,"['', 'any morning between 10 and 11:30']",,,"((10, and, 11:30, '),)"
8,Re: High Speed Internet Access,"['', '1. login: pallen pw: ke9davis', '', "" I...",,,"((ISP), (2), (IP), (DNS), (3))"
9,FW: fixed forward or other Collar floor gas pr...,"['', '---------------------- Forwarded by Phil...",,,"((Phillip, K, Allen, /, HOU, /, ECT), (01:42, ..."


In [ ]:
text = str(df_emails['summary_TFIDF'][32])
text="""The human coronavirus was first diagnosed in 1965 by Tyrrell and Bynoe from the respiratory tract sample of an adult with a common cold cultured on human embryonic trachea.1 Naming the virus is based on its crown-like appearance on its surface.2 Coronaviruses (CoVs) are a large family of viruses belonging to the Nidovirales order, which includes Coronaviridae, Arteriviridae, and Roniviridae families.3 Coronavirus contains an RNA genome and belongs to the Coronaviridae family.4 This virus is further subdivided into four groups, ie, the α, β, γ, and δ coronaviruses.5 α- and β-coronavirus can infect mammals, while γ- and δ- coronavirus tend to infect birds.6 Coronavirus in humans causes a range of disorders, from mild respiratory tract infections, such as the common cold to lethal infections, such as the severe acute respiratory syndrome (SARS), Middle East respiratory syndrome (MERS) and Coronavirus disease 2019 (COVID-19). The coronavirus first appeared in the form of severe acute respiratory syndrome coronavirus (SARS-CoV) in Guangdong province, China, in 20027 followed by Middle East respiratory syndrome coronavirus (MERS-CoV) isolated from the sputum of a 60-year-old man who presented symptoms of acute pneumonia and subsequent renal failure in Saudi Arabia in 2012.8 In December 2019, a β-coronavirus was discovered in Wuhan, China. The World Health Organization (WHO) has named the new disease as Coronavirus disease 2019 (COVID-19), and Coronavirus Study Group (CSG) of the International Committee has named it as SARS-CoV-2.9,10 Based on the results of sequencing and evolutionary analysis of the viral genome, bats appear to be responsible for transmitting the virus to humans"""
print(text)

The human coronavirus was first diagnosed in 1965 by Tyrrell and Bynoe from the respiratory tract sample of an adult with a common cold cultured on human embryonic trachea.1 Naming the virus is based on its crown-like appearance on its surface.2 Coronaviruses (CoVs) are a large family of viruses belonging to the Nidovirales order, which includes Coronaviridae, Arteriviridae, and Roniviridae families.3 Coronavirus contains an RNA genome and belongs to the Coronaviridae family.4 This virus is further subdivided into four groups, ie, the α, β, γ, and δ coronaviruses.5 α- and β-coronavirus can infect mammals, while γ- and δ- coronavirus tend to infect birds.6 Coronavirus in humans causes a range of disorders, from mild respiratory tract infections, such as the common cold to lethal infections, such as the severe acute respiratory syndrome (SARS), Middle East respiratory syndrome (MERS) and Coronavirus disease 2019 (COVID-19). The coronavirus first appeared in the form of severe acute respi

In [ ]:
# import spacy
# from spacy import displacy
# nlp = spacy.load('en_core_web_sm')
# spacy_stopwords = nlp.Defaults.stop_words
# stopwords = spacy_stopwords
# print(spacy_stopwords)

# Data cleaning

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
import re

def data_clean (tokens):
# Suppression des caractères spéciaux ou inutiles \"',]
#  print("Data cleaning --\n")
#  print("Avant :\n",type(tokens)," ",tokens)
  r = re.compile(r"[\\\''\",]")
  texte=r.sub('', tokens)
#  texte = sent_tokenize(texte)  # découpe en phrases stockées dans une liste
#  print("Après sent_tokenize :\n",type(texte)," ",texte)
  return texte

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
def remove_stop_words(text,stopwords):
  lst=[]
#  for token in str(text).split():
  for token in text.split():
    if token.lower() not in stopwords:    #checking whether the word is not 
      lst.append(token)                    #present in the stopword list.      
#    else:
#      print("suppression du stopword : ",token)
  # Join items in the list
#   print("Original text before removing stopwords  :   \n",text)
  result = ' '.join(lst)
#   print("Text after removing stopwords  :   \n",result)
  return result

In [ ]:
# Transforme Text en Dictionnaire - Key = indice de phrase - Value = Mots de la phrase
import re
from nltk.tokenize import sent_tokenize,word_tokenize

def dictionarize(article):
    dico={}
    phrases=sent_tokenize(article)
    phr2=[]
    for i, phr in enumerate(phrases):
        import re
        phr=re.sub(pattern ="[^a-zA-Z]", repl = ' ', string = phr)
        phr2.append(phr)
    
        phr=word_tokenize(phr)
        phr_lower=[w.lower() for w in phr]
        if len(phr_lower)>4:      
            dico[i]=phr_lower    
    return dico



In [ ]:
df_emails.shape[0]

200

# Summary methodS

In [ ]:
import pandas as pd
import numpy as np


def TF(token, artikle):
    """
    Calcule le score TF d'un mot dans un artikle
    
    token : Mot dont le score TF doit être calculé.
    
    artikle : Dictionnaire généré à partir d'un texte.
    """
    f = 0
    for key in artikle:
        for work in artikle[key]:
            if work == token:
                f += 1 
    return np.log(f+1)

def IDF(token, corpus):
    """
    Calcule le score IDF d'un mot dans un corpus d'artikles.
    
    token : Mot dont le score IDF doit être calculé.
    
    corpus : Liste d'artikles.
    """
    N = len(corpus)
    d=0
    present = False
    
    for artikle in corpus:
        for key in artikle:
            if token in artikle[key]:
                present = True
        d += int(present)
        present = False
                
    return np.log(N/(d+1) +1)

def TFIDF(token, artikle, corpus):
    """
    Calcule le score TF-IDF d'un mot dans un texte.
    
    token : mot dont le score doit être calculé.
    
    artikle : artikle qui servira à calculer le score du mot dans cet artikle.
    
    corpus : Liste d'artikles formant le corpus.
    """
    return TF(token, artikle)*IDF(token, corpus)

def score_sentence(corpus, artikle, sentence):
    """
    Calcule le score d'une phrase.
    
    corpus : Liste d'artikles.
    
    artikle : Dictionnaire de phrases.
    
    sentence : Phrase sous forme de liste de mots.
    """
    score_sentence = []
    for word in sentence :
        score_word = TFIDF(word, artikle, corpus)
        score_sentence.append(score_word)
    return np.mean(score_sentence)

def best_sentences(scores_artikle, nb_sentences):
    """
    Retourne les indices des phrases les plus importantes en fonction des scores obtenus.
    
    scores_artikle : Liste des scores de chaque phrase dans un texte.
    
    nb_sentences : Nombre de phrases à sélectionner.
    """
    
    return sorted(np.argsort(scores_artikle)[-nb_sentences:])

def summarize_TFIDF(i, n_sentences, df):
    """
    Synthèse extractive d'un article par la méthode TF-IDF.
    
    i : indice de l'article dans le corpus.
    
    n_sentences : nombre de phrases à conserver.
    
    df : DataFrame contenant les artikles dans une colonne 'Artikle'.    
    """
    corpus = df['body_dict']
    artikle = corpus[i]
    texte=df['body'][i]
    
    if len(corpus) <= n_sentences:
      print("Longeur corpus inférieure au nombre minimal de phrases retenu pour le résumé")
      return texte
    # Calcul du score de chaque phrase de l'artikle
    scores_artikle = [score_sentence(corpus, artikle, sentence) for sentence in artikle.values()]
 
    
    # Extraction des indices des phrases ayant les meilleurs scores
    result = best_sentences(scores_artikle, n_sentences)
#    print('best sentences :',result)
    
    # Séparation de phrases l'article original 
    tokenized_article = sent_tokenize(texte)
    
    # Extraction des phrases les plus importantes
    summary = [tokenized_article[i] for i in result]
#    print("[",i,"] :",summary)
    # transformation finale en chaine de caracteres
    texte = ''.join(summary)
    return texte

def summarize_spacy (text,ratio):
  # https://www.numpyninja.com/post/text-summarization-through-use-of-spacy-library
  nlp = spacy.load('en_core_web_sm')
  doc= nlp(text)
  tokens=[token.text for token in doc]
#  print("Tokens : \n",tokens)

  # calcul frequence de mots
  word_frequencies={}
  for word in doc:
    if word.text.lower() not in stopwords:
      if word.text.lower() not in punctuation:
        if word.text not in word_frequencies.keys():
          word_frequencies[word.text] = 1
        else:
          word_frequencies[word.text] += 1
#  print("\n word_frequencies : ",word_frequencies)

  # normalisation des frequence de mots
  max_frequency=max(word_frequencies.values())
  for word in word_frequencies.keys():
    word_frequencies[word]=word_frequencies[word]/max_frequency
#  print("\n Normalized word_frequencies : \n",word_frequencies)
  # Sentences token
  sentence_tokens= [sent for sent in doc.sents]
#  print(sentence_tokens)
  # Calculate the most important sentences by adding the word frequencies in each sentence.
  sentence_scores = {}
  for sent in sentence_tokens:
    for word in sent:
      if word.text.lower() in word_frequencies.keys():
        if sent not in sentence_scores.keys():                            
          sentence_scores[sent]=word_frequencies[word.text.lower()]
        else:
          sentence_scores[sent]+=word_frequencies[word.text.lower()]

  # identifier % du texte avec score maximum
  from heapq import nlargest
  select_length=int(len(sentence_tokens)*ratio)
  select_length
  summary=nlargest(select_length, sentence_scores,key=sentence_scores.get)
  final_summary=[word.text for word in summary]
  summary=''.join(final_summary)
  return summary

def summarize_abstractive (text, min_length, max_length):
  print("paramètres :", min_length, max_length)
  print("texte : ",len(text),text)
  from transformers import pipeline
#  print("Avant : ",text)
  # use the BART model, which is trained on the CNN/Daily Mail News Dataset, you can directly use the default parameters
  summarizer = pipeline("summarization")
  summarized = summarizer(text, min_length, max_length)
  summary=' '.join([str(i) for i in summarized])
#  print("Après : ",summary)
  return summary

# Test Unitaire

In [ ]:
id = 11
df_emails['body_clean'][id] = data_clean(df_emails['body'].iloc[id])

text = df_emails['body_clean'][id]
print("Cleaned text : ",type(text),len(text)," car. : ")
print(text,"\n")
# Extractive summary using TF-IDF
# ------------------------------------------------------------------------
# suppression de stopwords pour TFIDF
text = remove_stop_words(text,stopwords)

# Dictionnarize the text to be used with TF-IDF
print("Extractive summary using TF-IDF")
df_emails['body_dict'][id] = dictionarize(text)   # sans stopwords
if len(df_emails['body_dict'][id]) != 0:
  text = df_emails['body_dict'][id]
  # on affiche phrase par phrase le contenu de chaque mail
  for cle, valeur in text.items():
    print("Ligne ", cle, " : ", valeur)
  # Identification des n_sent les plus significatives
  max_sent = 3 
  df_emails['summary_TFIDF'][id] = summarize_TFIDF(id, max_sent, df_emails)
  ratio = len(df_emails['summary_TFIDF'][id])/len(df_emails['body_clean'][id])
  print("\nRésumé TFIDF - Len ",len(df_emails['summary_TFIDF'][id])," Ratio : ",int(ratio*100),"%")
  print(df_emails['summary_TFIDF'][id])

# ------------------------------------------------------------------------
# Extractive summary using Spacy
print("Extractive summary using Spacy")
# ratio = 0.20
text = df_emails['body_clean'].iloc[id]
df_emails['summary_spacy'][id] = summarize_spacy (str(text),ratio)
ratio = len(df_emails['summary_spacy'][id])/len(df_emails['body_clean'][id])
print("\nRésumé Spacy - Len ",len(df_emails['summary_spacy'][id]),"Ratio : ",int(ratio*100),"%")
print(df_emails['summary_spacy'][id])

# ------------------------------------------------------------------------
# Extractive summary using Transformers
print("\nExtractive summary using Transformers")
text = df_emails['body_clean'].iloc[id]
try:
  df_emails['summary_abstractive'][id] = summarize_abstractive (text, min_length=30, max_length=100)
  ratio = len(df_emails['summary_abstractive'][id])/len(df_emails['body_clean'][id])
  print("\nRésumé Transformers BERT  - Len ",len(df_emails['summary_abstractive'][id])," Ratio : ",int(ratio*100),"%")
  print(df_emails['summary_abstractive'][id])
except:
  print("Pb extractive summary on Id ",id)
  df_emails['summary_abstractive'][id] = "Erreur"

Cleaned text :  <class 'str'> 306  car. : 
[ Lucy   Here are the rentrolls:     Open them and save in the rentroll folder.  Follow these steps so you dont  misplace these files.   1.  Click on Save As  2.  Click on the drop down triangle under Save in:  3.  Click on the  (C): drive  4.  Click on the appropriate folder  5.  Click on Save:  Phillip] 

Extractive summary using TF-IDF
Ligne  0  :  ['lucy', 'rentrolls', 'open', 'save', 'rentroll', 'folder']
Ligne  1  :  ['follow', 'steps', 'dont', 'misplace', 'files']
Ligne  4  :  ['click', 'drop', 'triangle', 'save', 'in']

Résumé TFIDF - Len  170  Ratio :  55 %
['', 'Lucy,', '', ' Here are the rentrolls:', '', '', '', " Open them and save in the rentroll folder.Follow these steps so you don't ", 'misplace these files.', '', ' 1.
Extractive summary using Spacy

Résumé Spacy - Len  170 Ratio :  55 %
 Click on the  (C): drive  4. Follow these steps so you dont  misplace these files. Click on the drop down triangle under Save in:  3. Click on

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 (https://huggingface.co/sshleifer/distilbart-cnn-12-6)


Downloading:   0%|          | 0.00/1.76k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.14G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Ignoring args : (30, 100)
Your max_length is set to 142, but you input_length is only 95. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=47)



Résumé Transformers BERT  - Len  230  Ratio :  75 %
{'summary_text': ' Follow these steps so you dont  misplace these files . Click on the drop down triangle under Save in:  Click on  (C: drive) Click on (C): drive to save:  Phillip. Click on Save As:  Lucy. Lucy. Phillip. Lucy .'}


In [ ]:
id, df_emails['body'][id]

In [39]:
id, df_emails['summary_spacy'][id]

(156, '')

# Boucle de traitement

In [ ]:
deb = 0
fin = df_emails.shape[0]
print("Traitement en cours pour ",fin, "records")
#fin = 12
for id in range(deb,fin):
  print("\n id : ",id)
  print('--------------------')
  # coller quand OK pour les tests unitaires
  df_emails['body_clean'][id] = data_clean(df_emails['body'].iloc[id])
  text = df_emails['body_clean'][id]
  print("Cleaned text : ",len(str(text))," car.\n")
  print(text)
  # Extractive summary using TF-IDF
  # ------------------------------------------------------------------------
  # suppression de stopwords pour TFIDF
  text = remove_stop_words(text,stopwords)

  # Dictionnarize the text to be used with TF-IDF
  df_emails['body_dict'][id] = dictionarize(text)   # sans stopwords
  if len(df_emails['body_dict'][id]) != 0:
    text = df_emails['body_dict'][id]
    # on affiche phrase par phrase le contenu de chaque mail
    for cle, valeur in text.items():
      print("Ligne ", cle, " : ", valeur)
    # Identification des n_sent les plus significatives
    max_sent = 3 
    df_emails['summary_TFIDF'][id] = summarize_TFIDF(id, max_sent, df_emails)
    ratio = len(df_emails['summary_TFIDF'][id])/len(df_emails['body_clean'][id])
    print("\nRésumé TFIDF - Len ",len(df_emails['summary_TFIDF'][id])," Ratio : ",int(ratio*100),"%")
    print(df_emails['summary_TFIDF'][id])

# ------------------------------------------------------------------------
  # Extractive summary using Spacy
  ratio = 0.20
  text = df_emails['body_clean'].iloc[id]
  df_emails['summary_spacy'][id] = summarize_spacy (str(text),ratio)
  ratio = len(df_emails['summary_spacy'][id])/len(df_emails['body_clean'][id])
  print("\nRésumé Spacy - Len ",len(df_emails['summary_spacy'][id]),"Ratio : ",int(ratio*100),"%")
  print(df_emails['summary_spacy'][id])

# ------------------------------------------------------------------------
# Extractive summary using Transformers
  try:
    text = df_emails['body_clean'].iloc[id]
    df_emails['summary_abstractive'][id] = summarize_abstractive (text, min_length=30, max_length=100)
    ratio = len(df_emails['summary_abstractive'][id])/len(df_emails['body_clean'][id])
    print("\nRésumé Transformers BERT  - Len ",len(df_emails['summary_abstractive'][id])," Ratio : ",int(ratio*100),"%")
    print(df_emails['summary_abstractive'][id])
  except:
    print("Pb extractive summary on Id ",id)
    df_emails['summary_abstractive'][id] = "Erreur"


Traitement en cours pour  200 records

 id :  0
--------------------
Cleaned text :  26  car.

[ Here is our forecast   ]


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 (https://huggingface.co/sshleifer/distilbart-cnn-12-6)



Résumé Spacy - Len  0 Ratio :  0 %

paramètres : 30 100
texte :  26 [ Here is our forecast   ]


Ignoring args : (30, 100)
Your max_length is set to 142, but you input_length is only 10. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=5)



Résumé Transformers BERT  - Len  302  Ratio :  1161 %
{'summary_text': " The weather is expected to be mild in the next few days . It will be colder than normal in the early hours of next week . The weather should be mild enough to make it feel like it's warmer than normal for the rest of the year . For more weather coverage, visit CNN.com/suspective ."}

 id :  1
--------------------
Cleaned text :  787  car.

[ Traveling to have a business meeting takes the fun out of the trip.  Especially if you have to prepare a presentation.  I would suggest holding the business plan meetings here then take a trip without any formal business meetings.  I would even try and get some honest opinions on whether a trip is even desired or necessary.  As far as the business meetings I think it would be more productive to try and stimulate discussions across the different groups about what is working and what is not.  Too often the presenter speaks and the others are quiet just waiting for their turn.  

...

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 (https://huggingface.co/sshleifer/distilbart-cnn-12-6)



Résumé Spacy - Len  0 Ratio :  0 %

paramètres : 30 100
texte :  70 [ is your voice healed or are you going to use a real time messenger?]


Ignoring args : (30, 100)
Your max_length is set to 142, but you input_length is only 18. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=9)



Résumé Transformers BERT  - Len  246  Ratio :  351 %
{'summary_text': ' [ is your voice healed or are you going to use a real time messenger?] [ is you going for a real-time messenger? "Yes" [ is it a good idea to use your voice or not to use it? Please email us at www.mailmail.com/voice-injury .'}

 id :  156
--------------------
Cleaned text :  5  car.

[ no]


ValueError: ignored

# Sauvegarde du fichier final en csv

In [42]:
# SAUVEGARDE en csv du résultat.
file_name = "/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/enron_unique_output.csv"
df[deb:fin].to_csv(file_name, encoding='utf-8', index=False)
file_name = "/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/enron_unique_output.xls"
df[deb:fin].to_excel(file_name, encoding='utf-8', index=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  """


# SPACY - Summary (Abstractive / Word Embedding)

In [ ]:
#!pip install spacy-transformers

In [ ]:
# https://spacy.io/usage/embeddings-transformers
# https://spacy.io/api/transformer

In [ ]:
# https://manusidtech.com/index.php/2021/08/14/complete-nlp-web-app-using-spacy-and-hugging-face-transformers/
#!pip install datasets transformers[sentencepiece]
text = str(df_emails['body'][32])
text="""The human coronavirus was first diagnosed in 1965 by Tyrrell and Bynoe from the respiratory tract sample of an adult with a common cold cultured on human embryonic trachea.1 Naming the virus is based on its crown-like appearance on its surface.2 Coronaviruses (CoVs) are a large family of viruses belonging to the Nidovirales order, which includes Coronaviridae, Arteriviridae, and Roniviridae families.3 Coronavirus contains an RNA genome and belongs to the Coronaviridae family.4 This virus is further subdivided into four groups, ie, the α, β, γ, and δ coronaviruses.5 α- and β-coronavirus can infect mammals, while γ- and δ- coronavirus tend to infect birds.6 Coronavirus in humans causes a range of disorders, from mild respiratory tract infections, such as the common cold to lethal infections, such as the severe acute respiratory syndrome (SARS), Middle East respiratory syndrome (MERS) and Coronavirus disease 2019 (COVID-19). The coronavirus first appeared in the form of severe acute respiratory syndrome coronavirus (SARS-CoV) in Guangdong province, China, in 20027 followed by Middle East respiratory syndrome coronavirus (MERS-CoV) isolated from the sputum of a 60-year-old man who presented symptoms of acute pneumonia and subsequent renal failure in Saudi Arabia in 2012.8 In December 2019, a β-coronavirus was discovered in Wuhan, China. The World Health Organization (WHO) has named the new disease as Coronavirus disease 2019 (COVID-19), and Coronavirus Study Group (CSG) of the International Committee has named it as SARS-CoV-2.9,10 Based on the results of sequencing and evolutionary analysis of the viral genome, bats appear to be responsible for transmitting the virus to humans"""
print(text)

print(summarize_abstractive (text))
print(summarize_spacy (text))
print()

A voir

In [ ]:
# https://www.machinelearningplus.com/nlp/text-summarization-approaches-nlp-example/
# https://towardsdatascience.com/abstractive-summarization-using-pytorch-f5063e67510
# https://www.analyticsvidhya.com/blog/2021/10/text-summarization-using-the-conventional-hugging-face-transformer-and-cosine-similarity/


# Archived